In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import relu
from DataLoader import DataLoader
from halfkp import get_halfkp_indeicies
import chess
from chess import Board

import numpy as np

In [6]:
training_generator = DataLoader(128, "train")
val_generator = DataLoader(128, "val")

In [3]:
from keras import backend as K
def clipped_relu(x):
    return K.relu(x, max_value=1)

In [ ]:
halfAcc = Dense(256, clipped_relu)
i1 = tf.keras.Input((64*64*12,))
i2 = tf.keras.Input((64*64*12,))
acc1 = halfAcc(i1)
acc2 = halfAcc(i2)
layerStack = tf.keras.layers.Concatenate()([acc1, acc2])

h1 = Dense(32, clipped_relu)
h2 = Dense(32, clipped_relu)
out = Dense(1, 'sigmoid')

layerStack = out(h2(h1(layerStack)))

model = tf.keras.Model(inputs=[i1, i2], outputs=layerStack)

model.compile(optimizer=tf.keras.optimizers.Adam(), loss='mse', metrics=['mae'])

In [3]:
model = tf.keras.models.load_model("production/")

2022-03-27 11:10:09.039289: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-27 11:10:09.042673: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-27 11:10:09.044882: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-27 11:10:09.045480: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [5]:
#921/921 [==============================] - 24s 26ms/step - loss: 0.1499 - mae: 0.3252 - val_loss: 0.1661 - val_mae: 0.3544
#921/921 [==============================] - 25s 27ms/step - loss: 0.1361 - mae: 0.3087 - val_loss: 0.1685 - val_mae: 0.3563
model.fit(training_generator, validation_data=val_generator, epochs=1)

2981/2981 [==============================] - 130s 44ms/step - loss: 0.1439 - mae: 0.3189 - val_loss: 0.1613 - val_mae: 0.3482


In [19]:
indicies = get_halfkp_indeicies(Board("3R4/2P2kp1/b1r2p1p/1p5P/p4B2/P4P2/6PK/8 b - - 77 81"))
indicies = indicies.reshape(2, -1)
indicies = (np.array([indicies[0], indicies[0]]), np.array([indicies[1], indicies[1]]))
pred = model.predict(indicies)
print(pred[0])

[0.7542669]


In [8]:
pred = model.predict(val_generator[0][0]) * 2 - 1
act = val_generator[0][1] * 2 - 1
for i in range(128):
    print(pred[i], act[i], val_generator.getfen(0)[i])

[0.19559193] 1.0 8/5k1p/1q2b1p1/4P3/8/2QPP2P/1P4PK/8 w - - 1 42
[0.06666791] 0.0 2rq1k2/R4pp1/3p4/3Pp3/2B2B1b/1P3P2/1P1Q4/3K4 b - - 0 25
[0.06795657] -1.0 2r5/p1p2pk1/1pB1rnpp/2P1p3/8/P3PP1q/1PQ2P1P/K2R2R1 w - - 3 24
[0.11350274] 1.0 r2q1rk1/2pbbppp/p1np1n2/1p6/3PP3/NB3N1P/PP3PP1/R1BQR1K1 b - - 0 12
[0.50853384] 1.0 3R4/2P2kp1/b1r2p1p/1p5P/p4B2/P4P2/6PK/8 b - - 77 81
[-0.41439885] -1.0 1k6/8/5p1P/8/8/5np1/r7/5K2 w - - 0 61
[-0.02499861] -1.0 2r3k1/1b1r3p/1p1pp3/pPq1np2/P1P5/1B6/2Q2PPP/2R1RNK1 w - - 0 29
[0.04379761] 1.0 r1bq1rk1/1p1nnpbp/6p1/pPp1p3/P1Pp4/3P1NP1/1B1NPPBP/R2Q1RK1 b - - 0 13
[0.43809354] 1.0 8/1p2b1rk/p3p2p/8/P3N3/3R1P1P/4Q1PK/2q5 w - - 8 50
[-0.14248043] -1.0 2r2r1k/p1q3pp/1p2p3/3p1p2/b2P1P2/1RN1PBP1/2R4P/3Q2K1 w - - 0 23
[0.11118567] -1.0 r1b1k2r/pp1n1ppp/5n2/q1pp2B1/1b1P4/2NBPN2/PPQ2PPP/R3K2R b KQkq - 3 9
[-0.28942424] -1.0 8/2R4p/3p2pk/p2P1p2/1p4PP/q4B2/6K1/8 b - - 0 41
[0.25974047] 1.0 3r2k1/pqn2rpp/1p6/3p4/8/1P1Q2P1/PB2PP1P/2RR2K1 w - - 3 30
[0.14871323] 0.0 2rr2k1/

In [8]:
import time
model.save("models/model-{}".format(int(time.time())))

2022-03-27 11:00:00.900905: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: models/model-1648393200/assets
